1. 港股的汇率问题;
2. 使用不复权的数据和除权除息的数据,定点复权

### 港股的汇率问题

In [2]:
# 首先是拿到HKD/USD的数据
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import talib as ta
import datetime, time
import pickle
from tqdm import tqdm_notebook

import pdblp as pb
import blpapi

con = pb.BCon(port=8194, timeout=5000)
# con.start()

code = 'USDHKD BGNL Curncy'
start = '20100101'
end = '20190117'
# df = con.bdh(code, ['PX_LAST'], start, end)

In [4]:
with open('data/df_cur', 'rb') as f:
    df = pickle.load(f)
df.head()

ticker,USDHKD BGNL Curncy
field,PX_LAST
date,
2010-01-01,7.7552
2010-01-04,7.7554
2010-01-05,7.7569
2010-01-06,7.7547
2010-01-07,7.7546


In [5]:
df.tail()

ticker,USDHKD BGNL Curncy
field,PX_LAST
date,
2019-01-11,7.8394
2019-01-14,7.8425
2019-01-15,7.8433
2019-01-16,7.8431
2019-01-17,7.8432


```
ticker	USDHKD BGNL Curncy
field	PX_LAST
date	
2019-01-21	7.8448
2019-01-22	7.8455
2019-01-23	7.8462
2019-01-24	7.8458
2019-01-25	7.8466
```

In [7]:
df[code].head()

field,PX_LAST
date,
2010-01-01,7.7552
2010-01-04,7.7554
2010-01-05,7.7569
2010-01-06,7.7547
2010-01-07,7.7546


In [8]:
# 然后对港股的行情数据进行处理
from utils import *

In [9]:
data = ol_data.copy()

In [10]:
hk_code = df_code.query('region == "HK Equity"').symbol.unique().tolist()

In [14]:
# hk_code

In [25]:
df.index[0].strftime('%Y-%m-%d')

'2010-01-01'

In [13]:
data.loc[(slice(None), hk_code),:].head()

high    open     low     volume  Adj Close
date       code                                              
2010-01-04 1149   0.615   0.605   0.530  5244000.0      0.558
           1385   1.583   1.483   1.465  1420000.0      1.574
           148   18.469  18.171  17.725   966247.0     17.934
           1686   0.336   0.319   0.319  8601612.0      0.332
           1888   3.295   3.150   3.110  6502000.0      3.254

In [30]:
df.iloc[0][0]

7.7552

In [35]:
# 指定日期
for i in df.index[:5]:
    time_str = i.strftime('%Y-%m-%d')
#     data.loc[(time_str, hk_code),:] = data.loc[(time_str, hk_code),:] / df[i][0]
    print((data.loc[(time_str, hk_code),:] / df.loc[i][0]).head())
#     print(data.loc[(time_str, hk_code),:].head() )

# save data

Empty DataFrame
Columns: [high, open, low, volume, Adj Close]
Index: []
                     high      open       low        volume  Adj Close
date       code                                                       
2010-01-04 1149  0.079300  0.078010  0.068339  6.761740e+05   0.071950
           1385  0.204116  0.191222  0.188901  1.830982e+05   0.202955
           148   2.381437  2.343013  2.285504  1.245902e+05   2.312453
           1686  0.043325  0.041133  0.041133  1.109113e+06   0.042809
           1888  0.424865  0.406169  0.401011  8.383836e+05   0.419579
                     high      open       low         volume  Adj Close
date       code                                                        
2010-01-05 1149  0.074385  0.073096  0.070647  298572.883497   0.074385
           1385  0.209878  0.205237  0.199306  242364.862252   0.207557
           148   2.384844  2.315745  2.315745  132032.899741   2.373242
           1686  0.043316  0.043316  0.042414  496362.722222   0.042801

### 使用不复权的数据和除权除息的数据,定点复权

#### 拿到不复权的数据

```python
# 修改utils_update.py
temp_df = con.bdh(temp_list, fields, start, end)
# 改为
elements = [('CshAdjAbnormal', False), ('CapChg', False), ('CshAdjNormal', False)]
temp_df = con.bdh(temp_list, fields, start, end, elms=elements)
```

#### 获取除权除息的数据

```excel
=BDS("AAPL US Equity","DVD_HIST_ALL","headers=y","cols=7;rows=67")
```

In [15]:
code = "AAPL US Equity"
fields = "DVD_HIST_ALL"
df_dvd = con.bulkref(code, fields)
df_dvd

In [ ]:
code = "AAPL US Equity"
fields = "DVD_HIST_ALL"
overrides = {'headers':True}
df_dvd_2 = con.bulkref(code, fields, ovrds=overrides)
df_dvd_2

成功拿到这些数据之后，就是要考虑定点复权的问题

还有数据更新的问题，以及他想加入新的code和etf的问题

other bugs

In [1]:
%load_ext autoreload
%autoreload 2
from utils_v2 import *

In [37]:
res1 = calc_pair_trading('SMH', period=245, rt_period=255)
res2 = calc_pair_trading('SMH', period=1, rt_period=255)

In [38]:
res1['long_return'].sum().sum()

-0.11748640146501806

In [39]:
res2['long_return'].sum().sum()

-0.15015685089676106

In [40]:
res1['long_short_return'].sum().sum()

-0.02789620492814109

In [41]:
res2['long_short_return'].sum().sum()

-0.09323710379162711

In [2]:
res3 = calc_pair_trading('PNQI', period=245, rt_period=255)

BKNG or IUGXX don't have data, please check
GOOG or IUGXX don't have data, please check
FB or IUGXX don't have data, please check
AMZN or IUGXX don't have data, please check
NFLX or IUGXX don't have data, please check
AABA or IUGXX don't have data, please check
EQIX or IUGXX don't have data, please check
EBAY or IUGXX don't have data, please check
BIDU or IUGXX don't have data, please check
TWTR or IUGXX don't have data, please check
JD or IUGXX don't have data, please check
VRSN or IUGXX don't have data, please check
NTES or IUGXX don't have data, please check
EXPE or IUGXX don't have data, please check
ANET or IUGXX don't have data, please check
MELI or IUGXX don't have data, please check
IAC or IUGXX don't have data, please check
CTRP or IUGXX don't have data, please check
CSGP or IUGXX don't have data, please check
AKAM or IUGXX don't have data, please check
GDDY or IUGXX don't have data, please check
YNDX or IUGXX don't have data, please check
TRIP or IUGXX don't have data, please

In [7]:
ol_data.loc[(slice('2015','2099'), slice(None)), :].tail()

high     open      low      volume  Adj Close
date       code                                                  
2019-01-17 9684  3430.00  3345.00  3315.00    767100.0    3420.00
           9697  2214.00  2179.00  2157.00    488900.0    2204.00
           9766  5040.00  4995.00  4940.00    295600.0    4955.00
           981      6.70     6.56     6.52   9717418.0       6.60
           992      5.53     5.50     5.43  41066809.0       5.49